In [ ]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)
library(ggraph)
library(igraph)

In [ ]:
df <- lapply(snakemake@input$info, function(x){
    return(fread(x))
})
zos <- unlist(lapply(df,nrow))
df2 <- bind_rows(df[zos!=0]) %>%
    group_by(donor, reciever, bin) %>%
    summarise(N=n()) %>%
    ungroup() %>%
    group_by(donor, reciever) %>%
    summarise(N=max(N), bins=paste(unlist(unique(bin)),collapse=",")) %>%
    ungroup() %>%
    filter(N > 2, donor != reciever)
df2

In [ ]:
# Convert your data frame to an igraph object
graph_data <- df2 %>%
  dplyr::select(donor, reciever, N) %>%  # Ensure you're only taking necessary columns
  graph_from_data_frame(directed = TRUE) # Create a directed graph


In [ ]:
options(repr.plot.width = 10, repr.plot.height = 6, jupyter.plot_mimetypes = "image/svg+xml")
ggraph(graph_data, layout = 'kk') +  # 'kk' is the Kamada-Kawai layout
  geom_edge_link(aes(width = N), 
                 edge_colour = "gray", 
                 alpha = 0.8, 
                 arrow = arrow(length = unit(4, 'mm'), type = 'closed', ends = 'last')) +
  geom_node_point(color = "darkblue", size = 5) +
  geom_node_text(aes(label = name), vjust = 1.8, hjust=-0.1, color = "black", check_overlap = T) +
  ggtitle("Directed Graph of Donor to Receiver with Arrows") +
  theme(legend.position = "bottom") #+
#theme(plot.margin = margin(1, 5, 1, 1, "cm")) +
#coord_cartesian(xlim=c(-2.2,2.2), ylim=c(-2,3))




In [ ]:
options(repr.plot.width = 8, repr.plot.height = 7, jupyter.plot_mimetypes = "image/svg+xml")
ggraph(graph_data, layout = 'kk') +
  geom_edge_link(aes(width = N), 
                 edge_colour = "gray", 
                 alpha = 0.8, 
                 arrow = arrow(length = unit(4, 'mm'), type = 'closed', ends = 'last')) +
  geom_node_point(color = "darkblue", size = 5) +
  geom_node_text(aes(label = name), vjust = 1.8, hjust=0.5, color = "black", check_overlap = TRUE) +
  theme_graph() +
  ggtitle("Directed Graph of Donor to Receiver with Arrows") +
  theme(legend.position = "top", plot.margin = margin(1, 1, 1, 1, "cm"))


In [ ]:
library(ggraph)
library(igraph)

# Adjusting the options to better suit the SVG output in Jupyter
options(repr.plot.width = 10, repr.plot.height = 6, jupyter.plot_mimetypes = "image/svg+xml")

# Assuming graph_data is already prepared as an igraph object
ggraph(graph_data, layout = 'linear', circular = T) +  # Using Fruchterman-Reingold layout
  # First layer: edges with less prominent color

  # Second layer: arrows in a different, more prominent color
  geom_edge_arc(aes(width = N),
                colour="dark green",
                alpha = 0.5, 
                arrow = arrow(length = unit(2, 'mm'), type = 'closed', ends = 'last'),
                curvature = 1.2, ) +
  # geom_edge_arc(aes(width = N), 
  #               colour = "green", 
  #               alpha = 0.5, 
  #               curvature = 0.1) +
  # geom_node_point(color = "darkblue", size = 5, alpha = 0.3) +
  geom_node_text(aes(label = name), vjust = 1.8, hjust=-0.1, color = "black", check_overlap = TRUE, size = 2) +
  theme(legend.position = "bottom",
        plot.background = element_rect(fill = "white", colour = NA),  # ensure plot background is white
        panel.background = element_rect(fill = "white", colour = NA))


In [ ]:
# Assuming graph_data is already an igraph object
# Example: Adding a custom edge attribute for directionality
E(graph_data)$type <- ifelse(runif(ecount(graph_data)) > 0.5, "Type1", "Type2")

# Use this 'type' attribute to set different curvatures


In [ ]:
graph_data

In [ ]:
library(ggraph)
library(igraph)

# Adjusting the options for better SVG output in Jupyter
options(repr.plot.width = 12, repr.plot.height = 5, jupyter.plot_mimetypes = "image/svg+xml")

# Re-evaluate the use of layout if x and y coordinates should be preserved 
p1 <- ggraph(graph_data, layout = 'linear') + 
  geom_edge_arc(aes(width = N),
                curvature = ifelse(E(graph_data)$type == "Type1", 1.0, -1.0),  # Apply curvature outside of aes()
                arrow = arrow(length = unit(3, 'mm'), type = 'closed', ends = 'last',), color = "red",  alpha = 0.5) +

  geom_edge_arc(aes(width = N),
                curvature = ifelse(E(graph_data)$type == "Type1", 1.0, -1.0),  # Apply curvature outside of aes()
                , color = "grey",  alpha = 0.5) +


  geom_node_text(aes(label = name), vjust = 0, hjust = 0.1, 
                 color = "black", check_overlap = F, size = 5,  repel = TRUE, force=200) +
  geom_node_point(color = "darkblue", size = 2, alpha = 0.5) +

  theme(legend.position = "bottom",
        plot.background = element_rect(fill = "white", colour = NA),  
        panel.background = element_rect(fill = "white", colour = NA),
  legend.text = element_text(size = 12),  # Increase legend text size
        legend.title = element_text(size = 14),  # Increase legend title siz       
       ) #+
#coord_cartesian(xlim=c(-1.5,1.5), ylim=c(-1.1,1.2))



In [ ]:
df3 <- df2 %>%
    select(reciever, bins) %>%
    distinct()
df3

In [ ]:
# Example dataframe
options(repr.plot.width = 8, repr.plot.height = 5, jupyter.plot_mimetypes = "image/svg+xml")

df3 <- data.frame(
  reciever = c("Pirellulales", "Rhodospirillales", "Flavobacteriales", 
               "Cytophagales", "Balneolales", "Burkholderiales", 
               "Planctomycetales", "Rhodobacterales", "Xanthomonadales", 
               "Hyphomicrobiales", "Maricaulales"),
  bins = c("bin.8", "bin.18", "bin.3", "bin.2", "bin.15", "bin.16", 
           "bin.13", "bin.11", "bin.14", "bin.1,bin.12,bin.5,bin.9", "bin.4")
)

# Creating a text plot
text_plot <- ggplot(df3, aes(x = 1, y = factor(reciever, levels = rev(df3$reciever)), label = paste(reciever, bins, sep = ": "))) +
  geom_text(hjust = 0, size = 3.5) +
  theme_void() +
  theme(
    plot.margin = unit(c(1, 1, 1, 1), "cm"),  # Add some padding
    panel.background = element_rect(fill = "white", colour = NA)  # Ensure white background
  ) +
  labs(x = NULL, y = NULL)  # Remove any labels
text_plot

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8, jupyter.plot_mimetypes = "image/svg+xml")

library(patchwork)
op <- p1|text_plot + plot_layout(widths = c(2, 1))

In [ ]:
ggsave(plot = op, filename = snakemake@output$plot, width = 16, height =8)
fwrite(df2,file=snakemake@output$csv)